In [2]:
import pandas as pd 
import requests
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = df[0]
df.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [7]:
code_list = df.values.tolist()

In [8]:
postal_code =[]
borough = []
nb = []
for i in code_list:
    for j in i:
        if j[3:] == 'Not assigned':
            pass
        else:
            postal_code.append(j[0:3])
            sep = j[3:].split('(')
            borough.append(sep[0])
            nb.append(sep[1].split(')')[0])
data = {'Postal_Code':postal_code,'Borough':borough,'Neighborhood':nb}

In [9]:
df_cp_can = pd.DataFrame(data)
df_cp_can.head()

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
df_cp_can.shape

(103, 3)

In [15]:
!pip install geocoder
import geocoder

df_coord = pd.read_csv('C:/Users/Marisa/Downloads/Geospatial_Coordinates.csv')
print(df_coord.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [16]:
df_coord.columns = ['Postal_Code','Latitude','Longitude']
df_coord.head()

,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_total = pd.merge(df_cp_can,df_coord)

df_total.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [30]:
import folium

In [32]:
toronto = folium.Map([43.70011, -79.4163],zoom_start=9)
toronto

In [36]:
torontob = df_total[df_total.Borough.str.contains('Toronto')]
torontob.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [38]:
for pc,b,n,lat,long in zip(torontob['Postal_Code'],
                           torontob['Borough'],
                           torontob['Neighborhood'],
                           torontob['Latitude'],
                           torontob['Longitude']):
    label ='{}, {}'.format(n,b)
    label = folium.Popup(label, parse_html=False)
    folium.CircleMarker([lat,long],
                        radius=5,
                        popup=label,
                        color='blue',
                        parse_html=True).add_to(toronto)
toronto